In [9]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit.compiler import transpile
from math import gcd
from fractions import Fraction

def c_amod15(a, power):
    """Controlled multiplication by a mod 15"""
    if a not in [2,7,8,11,13]:
        raise ValueError("'a' must be 2,7,8,11 or 13")
    U = QuantumCircuit(4)        
    for iteration in range(power):
        if a in [2,13]:
            U.swap(0,1)
            U.swap(1,2)
            U.swap(2,3)
        if a in [7,8]:
            U.swap(2,3)
            U.swap(1,2)
            U.swap(0,1)
        if a == 11:
            U.swap(1,3)
            U.swap(0,2)
        if a in [7,11,13]:
            for q in range(4):
                U.x(q)
    U = U.to_gate()
    U.name = f"{a}^{power} mod 15"
    c_U = U.control()
    return c_U

def qft_dagger(n):
    """n-qubit QFT dagger"""
    qc = QuantumCircuit(n)
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    qc.name = "QFT†"
    return qc

def shor_circuit(a):
    n_count = 8  # number of counting qubits
    n_state = 4  # number of qubits to represent the state

    qr_count = QuantumRegister(n_count, 'count')
    qr_state = QuantumRegister(n_state, 'state')
    cr_count = ClassicalRegister(n_count, 'c')

    qc = QuantumCircuit(qr_count, qr_state, cr_count)

    # Initialize counting qubits in superposition
    for qubit in range(n_count):
        qc.h(qr_count[qubit])

    # Initialize state in |1>
    qc.x(qr_state[0])

    # Do controlled-U operations
    for i, qubit in enumerate(qr_count):
        qc.append(c_amod15(a, 2**i), [qubit] + qr_state[:])

    # Do inverse QFT
    qc.append(qft_dagger(n_count), qr_count)

    # Measure
    qc.measure(qr_count, cr_count)

    return qc

def run_shor(N, a):
    qc = shor_circuit(a)
    
    # Use AerSimulator
    simulator = AerSimulator()
    
    # Transpile the circuit for the simulator
    transpiled_qc = transpile(qc, simulator)
    
    # Run the simulation
    job = simulator.run(transpiled_qc, shots=1000)
    result = job.result()
    counts = result.get_counts()
    
    # Plot the results
    plot_histogram(counts)
    
    measured_phases = []
    for output in counts:
        decimal = int(output, 2)  # Convert (base 2) string to decimal
        phase = decimal/(2**8)  # Find corresponding eigenvalue
        measured_phases.append(phase)
    
    phases = np.mean(measured_phases)
    frac = Fraction(phases).limit_denominator(N)
    r = frac.denominator
    
    print(f"Measured phase: {phases}")
    print(f"Estimated r: {r}")
    
    if r % 2 != 0:
        r *= 2
    
    guesses = [gcd(a**(r//2)-1, N), gcd(a**(r//2)+1, N)]
    return guesses

# Example: Factorize N = 15
N = 5900
a = 7
print(f"Attempting to factor N = {N} using a = {a}")
guesses = run_shor(N, a)
print(f"Guessed factors: {guesses}")

Attempting to factor N = 5900 using a = 7
Measured phase: 0.375
Estimated r: 8
Guessed factors: [100, 2]
